# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 58 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.03436
Retrieving document from  https://arxiv.org/e-print/2402.03498


/tmp/ipykernel_2083/3009462391.py:49: LatexWarning: 2402.03436 did not run properly
HTTP Error 500: Internal Server Error
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


/tmp/ipykernel_2083/3009462391.py:49: LatexWarning: 2402.03498 did not run properly
HTTP Error 500: Internal Server Error
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2402.03594


Retrieving document from  https://arxiv.org/e-print/2402.04130


/tmp/ipykernel_2083/3009462391.py:49: LatexWarning: 2402.03594 did not run properly
HTTP Error 500: Internal Server Error
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2083/3009462391.py:49: LatexWarning: 2402.04130 did not run properly
HTTP Error 500: Internal Server Error
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.03436-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.03436) | **Euclid preparation. Optical emission-line predictions of intermediate-z  galaxy populations in GAEA for the Euclid Deep and Wide Surveys**  |
|| E. Collaboration, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-02-07*|
|*Comments*| *32 pages, 15 figures, submitted to A&A*|
|**Abstract**| In anticipation of the Euclid Wide and Deep Surveys, we present optical emission-line predictions at intermediate redshifts from 0.4 to 2.5. Our approach combines a mock light cone from the GAEA semi-analytic model to self-consistently model nebular emission from HII regions, narrow-line regions of active galactic nuclei (AGN), and evolved stellar populations. Our analysis focuses on seven optical emission lines: H$\alpha$, H$\beta$, [SII]$\lambda\lambda 6717, 6731$, [NII]$\lambda 6584$, [OI]$\lambda 6300$, [OIII]$\lambda 5007$, and [OII]$\lambda\lambda 3727, 3729$. We find that Euclid will predominantly observe massive, star-forming, and metal-rich line-emitters. Interstellar dust, modelled using a Calzetti law with mass-dependent scaling, may decrease observable percentages by a further 20-30% with respect to our underlying emission-line populations from GAEA. We predict Euclid to observe around 30-70% of H$\alpha$-, [NII]-, [SII]-, and [OIII]-emitting galaxies at redshift below 1 and under 10% at higher redshift. Observability of H$\beta$-, [OII]-, and [OI]- emission is limited to below 5%. For the Euclid-observable sample, we find that BPT diagrams can effectively distinguish between different galaxy types up to around redshift 1.8, attributed to the bias toward metal-rich systems. Moreover, we show that the relationships of H$\alpha$ and [OIII]+H$\beta$ to the star-formation rate, and the [OIII]-AGN luminosity relation, exhibit minimal changes with increasing redshift. Based on line ratios [NII]/H$\alpha$, [NII]/[OII], and [NII]/[SII], we further propose novel z-invariant tracers for the black hole accretion rate-to-star formation rate ratio. Lastly, we find that commonly used metallicity estimators display gradual shifts in normalisations with increasing redshift, while maintaining the overall shape of local calibrations. This is in tentative agreement with recent JWST data. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 500: Internal Server Error</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.03498-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.03498) | **The TESS-Keck Survey. XIX. A Warm Transiting Sub-Saturn Mass Planet and  a non-Transiting Saturn Mass Planet Orbiting a Solar Analog**  |
|| M. L. Hill, et al. -- incl., <mark>E. Matthews</mark> |
|*Appeared on*| *2024-02-07*|
|*Comments*| *Accepted for publication in The Astronomical Journal. 15 pages, 7 figures*|
|**Abstract**| The Transiting Exoplanet Survey Satellite (TESS) continues to dramatically increase the number of known transiting exoplanets, and is optimal for monitoring bright stars amenable to radial velocity (RV) and atmospheric follow-up observations. TOI-1386 is a solar-type (G5V) star that was detected via TESS photometry to exhibit transit signatures in three sectors with a period of 25.84 days. We conducted follow-up RV observations using Keck/HIRES as part of the TESS-Keck Survey (TKS), collecting 64 RV measurements of TOI-1386 with the HIRES spectrograph over 2.5 years. Our combined fit of the TOI-1386 photometry and RV data confirm the planetary nature of the detected TESS signal, and provide a mass and radius for planet b of $0.148\pm0.019$ $M_J$ and $0.540\pm0.017$ $R_J$, respectively, marking TOI-1386 b as a warm sub-Saturn planet. Our RV data further reveal an additional outer companion, TOI-1386 c, with an estimated orbital period of 227.6 days and a minimum mass of $0.309\pm0.038$ $M_J$. The dynamical modeling of the system shows that the measured system architecture is long-term stable, although there may be substantial eccentricity oscillations of the inner planet due to the dynamical influence of the outer planet. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 500: Internal Server Error</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.03594-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.03594) | **Upgrading the GRAVITY fringe tracker for GRAVITY+: Tracking the white  light fringe in the non-observable Optical Path Length state-space**  |
|| M. Nowak, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-02-07*|
|*Comments*| *16 pages, 8 figures. Accepted for publication in A&A*|
|**Abstract**| Aims. As part of the ongoing GRAVITY+ upgrade of the Very Large Telescope Interferometer infrastructure, we aim to improve the performance of the GRAVITY Fringe-Tracker, and to enable its use by other instruments. Methods. We modify the group delay controller to consistently maintain tracking in the white light fringe, characterised by a minimum group delay. Additionally, we introduce a novel approach in which fringe-tracking is performed in the non-observable Optical Path Length state-space, using a covariance-weighted Kalman filter and an auto-regressive model of the disturbance. We outline this new state-space representation, and the formalism we use to propagate the state-vector and generate the control signal. While our approach is presented specifically in the context of GRAVITY/GRAVITY+, it can easily be adapted to other instruments or interferometric facilities. Results. We successfully demonstrate phase delay tracking within a single fringe, with any spurious phase jumps detected and corrected in less than 100 ms. We also report a significant performance improvement, as evidenced by a reduction of about 30 to 40% in phase residuals, and a much better behaviour under sub-optimal atmospheric conditions. Compared to what was observed in 2019, the median residuals have decreased from 150 nm to 100 nm on the Auxiliary Telescopes and from 250 nm to 150 nm on the Unit Telescopes. Conclusions. The improved phase-delay tracking combined with whit light fringe tracking means that from now-on, the GRAVITY Fringe-Tracker can be used by other instruments operating in different wavebands. The only limitation remains the need for an optical path dispersion adjustment. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 500: Internal Server Error</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.04130-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.04130) | **The Stellar "Snake" -- II: The Mass Function**  |
|| X.-M. Yang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-02-07*|
|*Comments*| *19 pages, 15 figures, Accepted for publication on MNRAS*|
|**Abstract**| We present a comprehensive investigation on the mass function (MF) of a snake-like stellar structure in the solar neighbourhood, building on our previous discovery. To ensure the reliability of the data, we reselect the member stars of the Stellar ``Snake'' in the latest {\it Gaia} Data Release 3 using the same approach as the initial series of articles. We also precisely measure the physical parameters of the clusters within the Stellar Snake. In light of the high completeness of the member stars in the cluster regions, we develop a simulated model color-magnitude diagram-based inference method to derive the mass function, binary fraction, and mass-ratio distribution of the clusters in the Stellar Snake. Notably, despite their similar ages and metallicity, we discover systematic variations in the MFs along the elongation direction of the Snake in the mass range of 0.5 to 2.0 M$_\odot$. The ``head'' of the Snake conforms to a canonical initial mass function with a power-law slope of $\alpha\sim-2.3$. Extending towards the ``tail,'' the MF becomes more top-light, indicating a deficiency of massive stars within these clusters. This result provides evidence for the delayed formation of massive stars in the clusters. Such clues give support to the hypothesis that the Stellar Snake constitutes as a hierarchically primordial structure. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 500: Internal Server Error</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

367  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
